In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

'''
найти дескрипторы
и
совместить

https://www.learnopencv.com/image-alignment-feature-based-using-opencv-c-python/

https://www.learnopencv.com/homography-examples-using-opencv-python-c/

https://github.com/vivekseth/blog-posts/tree/master/Difference-between-perspective-transform-homography-essential-matrix-fundamental-matrix

opencv getaffinetransform homography

opencv homography panorama python

https://stackoverflow.com/questions/11237948/findhomography-getperspectivetransform-getaffinetransform
https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html?highlight=getaffinetransform

https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#Mat%20findHomography(InputArray%20srcPoints,%20InputArray%20dstPoints,%20int%20method,%20double%20ransacReprojThreshold,%20OutputArray%20mask)

http://answers.opencv.org/question/4538/getaffinetransform-getperspectivetransform-or-findhomography/

'''

'\nнайти дескрипторы\nи\nсовместить\n\nhttps://www.learnopencv.com/image-alignment-feature-based-using-opencv-c-python/\n\nhttps://www.learnopencv.com/homography-examples-using-opencv-python-c/\n\nhttps://github.com/vivekseth/blog-posts/tree/master/Difference-between-perspective-transform-homography-essential-matrix-fundamental-matrix\n\nopencv getaffinetransform homography\n\nopencv homography panorama python\n\nhttps://stackoverflow.com/questions/11237948/findhomography-getperspectivetransform-getaffinetransform\nhttps://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html?highlight=getaffinetransform\n\nhttps://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#Mat%20findHomography(InputArray%20srcPoints,%20InputArray%20dstPoints,%20int%20method,%20double%20ransacReprojThreshold,%20OutputArray%20mask)\n\nhttp://answers.opencv.org/question/4538/getaffinetransform-getperspectivetransform-or-findhomography/\n\n'

In [3]:
noises = ['None','gauss', 's&p', 'poisson', 'speckle']

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[tuple(coords)] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[tuple(coords)] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy
    
    return image



# Matcher BRUTEFORCE

In [6]:
@interact_manual(noise=noises, rotate=(0, 360), scale=(0.5, 1.5, 1e-2),
          minHessian=(1, 50000, 1))
def features_1_detect(noise, rotate, scale,
                   minHessian):
    
    
    
   
    img1 = cv.imread('/Users/kolsha/Desktop/milk1.jpg')
    img2 = cv.imread('/Users/kolsha/Desktop/milk2.jpg')

    img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
    img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)
    img1_gray = cv.cvtColor(img1, cv.COLOR_RGB2GRAY)
    
    
    
    num_rows, num_cols = img1_gray.shape[:2]

    rotation_matrix = cv.getRotationMatrix2D((num_cols/2, num_rows/2), rotate, scale)
    
    
    img2 = noisy(noise, img2)
    img2 = img2.astype('uint8')
    
    
    img2 = cv.warpAffine(img2, rotation_matrix, (num_cols, num_rows))
    
    img2_gray = cv.cvtColor(img2, cv.COLOR_RGB2GRAY)
    
    #-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors
    detector = cv.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    
    #detector = cv.ORB_create(100)
    
    keypoints1, descriptors1 = detector.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = detector.detectAndCompute(img2_gray, None)
    #-- Step 2: Matching descriptor vectors with a brute force matcher
    # Since SURF is a floating-point descriptor NORM_L2 is used
    
    #matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    #matches = matcher.match(descriptors1, descriptors2)
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
    matches = matcher.knnMatch(descriptors1, descriptors2, 2)
    #matcher.knnMatch()
    
    #matches.sort(key=lambda x: x.distance, reverse=False)
    
    #numGoodMatches = int(len(matches) * 0.7)
    #matches = matches[:numGoodMatches]
    
    good_matches = []
    for m,n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)
            
    matches = good_matches
    
    #-- Draw matches
    img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
    cv.drawMatches(img1, keypoints1, img2, keypoints2, matches, img_matches)
    
    
      # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
        
        
    h, mask = cv.findHomography(points1, points2, cv.RANSAC) # устойчив 
    
    height, width, channels = img2.shape
    img1Reg = cv.warpPerspective(img1, h, (width, height))
    
    
    
    
    
    titles = ['img_matches', 'warped']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [img_matches,
             np.concatenate((img1Reg, img2), axis=1)
             ]#[src, tpl, dst_src, dst_tpl]
    #fig = plt.gcf()
    #fig.set_size_inches(20, 20)
    for i in range(len(images)):
        #plt.subplot(2, 3, i+1)
        plt.figure(figsize = (20,10))
        plt.imshow(images[i], aspect='auto')
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (rotate, scale)
    
    
    
    

interactive(children=(Dropdown(description='noise', options=('None', 'gauss', 's&p', 'poisson', 'speckle'), va…

# Matcher_FLANNBASED

In [4]:
@interact_manual(noise=noises, rotate=(0, 360), scale=(0.5, 1.5, 1e-2),
          minHessian=(1, 50000, 1), ratio_thresh=(0.1, 2.0, 0.1))
def features_2_detect(noise, rotate, scale,
                   minHessian, ratio_thresh):
    
    
    
    src = cv.imread('grabCut/bike.jpg')
    #src_gray = cv.medianBlur(src_gray, 5)
    src = cv.cvtColor(src, cv.COLOR_BGR2RGB)
    
    src_gray = cv.cvtColor(src, cv.COLOR_RGB2GRAY)
    
    img = src.copy()
    num_rows, num_cols = img.shape[:2]

    rotation_matrix = cv.getRotationMatrix2D((num_cols/2, num_rows/2), rotate, scale)
    
    
    img = noisy(noise, img)
    img = img.astype('uint8')
    
    
    img = cv.warpAffine(img, rotation_matrix, (num_cols, num_rows))
    
    img_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    
    detector = cv.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    #sift = cv.xfeatures2d.SIFT_create()
    
    keypoints1, descriptors1 = detector.detectAndCompute(src_gray, None)
    keypoints2, descriptors2 = detector.detectAndCompute(img_gray, None)
    
    #-- Step 2: Matching descriptor vectors with a FLANN based matcher
    # Since SURF is a floating-point descriptor NORM_L2 is used
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
    knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)
    #-- Filter matches using the Lowe's ratio test
    
    good_matches = []
    for m,n in knn_matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    #-- Draw matches
    img_matches = np.empty((max(src.shape[0], img.shape[0]), src.shape[1]+img.shape[1], 3), dtype=np.uint8)
    cv.drawMatches(src, keypoints1, img, keypoints2, good_matches, img_matches, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    
    
    
    
    titles = ['img_matches']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [img_matches]#[src, tpl, dst_src, dst_tpl]
    fig = plt.gcf()
    fig.set_size_inches(20, 20)
    for i in range(len(images)):
        #plt.subplot(2, 3, i+1)
        plt.figure(figsize = (20,10))
        plt.imshow(images[i], aspect='auto')
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (rotate, scale)
    
    
    
    

interactive(children=(Dropdown(description='noise', options=('None', 'gauss', 's&p', 'poisson', 'speckle'), va…